In [1]:
import pyspark

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [13]:
spark = SparkSession.builder.appName("test").config("spark.sql.autoBroadcastJoinThreshold",104857600).getOrCreate()

In [14]:
df = spark.read.format("csv").option("header",True).option("inferSchema",True).load("employees.csv")

In [15]:
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [16]:
data = [(50,"sales"),(40,"IT")]

In [17]:
schema = ["DEPARTMENT_ID","DEPT_NAME"]

In [18]:
sales_df = spark.createDataFrame(data=data,schema=schema)

In [20]:
final_df = df.join(broadcast(sales_df),df.DEPARTMENT_ID == sales_df.DEPARTMENT_ID,how="inner").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [cast(DEPARTMENT_ID#127 as bigint)], [DEPARTMENT_ID#196L], Inner, BuildRight, false
   :- Filter isnotnull(DEPARTMENT_ID#127)
   :  +- FileScan csv [EMPLOYEE_ID#117,FIRST_NAME#118,LAST_NAME#119,EMAIL#120,PHONE_NUMBER#121,HIRE_DATE#122,JOB_ID#123,SALARY#124,COMMISSION_PCT#125,MANAGER_ID#126,DEPARTMENT_ID#127] Batched: false, DataFilters: [isnotnull(DEPARTMENT_ID#127)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/fci/Desktop/Spark-Tutorials/employees.csv], PartitionFilters: [], PushedFilters: [IsNotNull(DEPARTMENT_ID)], ReadSchema: struct<EMPLOYEE_ID:int,FIRST_NAME:string,LAST_NAME:string,EMAIL:string,PHONE_NUMBER:string,HIRE_D...
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=153]
      +- Filter isnotnull(DEPARTMENT_ID#196L)
         +- Scan ExistingRDD[DEPARTMENT_ID#196L,DEPT_NAME#197]




In [ ]:

    # First, It read the parquet file and created a Larger DataFrame with limited records.
    # Then, BroadcastHashJoin is performed between the smallerDF and LargerDF using the condition provided.
    # Even if the smallerDF is not specified to be broadcasted in our code, Spark automatically broadcasts the smaller DataFrame into executor memory by default.
